<a href="https://colab.research.google.com/github/francodem/timesfm-bygoogle-demo/blob/main/testing_timesfm_from_google_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf timesfm
!git clone https://github.com/google-research/timesfm.git /content/timesfm_repo
!pip install -e /content/timesfm_repo

Cloning into '/content/timesfm_repo'...
remote: Enumerating objects: 1046, done.
remote: Counting objects: 100% (508/508), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 1046 (delta 364), reused 302 (delta 284), pack-reused 538 (from 2)
Receiving objects: 100% (1046/1046), 2.31 MiB | 8.74 MiB/s, done.
Resolving deltas: 100% (538/538), done.
Obtaining file:///content/timesfm_repo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for timesfm (pyproject.toml) ... done
  Created wheel for timesfm: filename=timesfm-2.0.0-py3-none-any.whl size=6539 sha256=4acf9f4512d9fdfe7afba9c9e2f577fe974cefe232965b76a4269d1389a8a15f
  Stored in directory: /tmp/pip-ephem-wheel-cache-z36c1s3f/wheels/83/b7/9b/0492f2f434826a82cac1e3200c715f9f20cea95e9b5c071cd5
Successfully built timesfm
  Attempting uninstall: 

In [ ]:
# LIMPIEZA
%pip uninstall -y timesfm
import shutil, os, sys, glob
for p in ["/content/timesfm", "/content/timesfm.py", "/content/timesfm/__pycache__"]:
    shutil.rmtree(p, ignore_errors=True)

# Instalar desde un folder que NO se llame "timesfm" (evita sombra)
!rm -rf /content/timesfm_repo
!git clone https://github.com/google-research/timesfm.git /content/timesfm_repo

# Instalación editable desde la RAÍZ DEL REPO (donde está pyproject.toml)
%pip install -e /content/timesfm_repo


Found existing installation: timesfm 2.0.0
Uninstalling timesfm-2.0.0:
  Successfully uninstalled timesfm-2.0.0
Cloning into '/content/timesfm_repo'...
remote: Enumerating objects: 1046, done.
remote: Counting objects: 100% (506/506), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 1046 (delta 363), reused 301 (delta 283), pack-reused 540 (from 2)
Receiving objects: 100% (1046/1046), 2.31 MiB | 8.91 MiB/s, done.
Resolving deltas: 100% (538/538), done.
Obtaining file:///content/timesfm_repo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for timesfm (pyproject.toml) ... done
  Created wheel for timesfm: filename=timesfm-2.0.0-py3-none-any.whl size=6539 sha256=4acf9f4512d9fdfe7afba9c9e2f577fe974cefe232965b76a4269d1389a8a15f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ezsnn_vq/wheels

In [ ]:
# Reiniciar runtime para limpiar import caches (obligatorio)
import os; os.kill(os.getpid(), 9)

In [ ]:
import timesfm, sys, pkgutil, os
print("timesfm.__file__:", getattr(timesfm, "__file__", None))
print("timesfm.__path__:", getattr(timesfm, "__path__", None))

# ¿Qué submódulos ve Python dentro de timesfm?
print("Submódulos disponibles:",
      [m.name for m in pkgutil.iter_modules(timesfm.__path__)])

# (Deberías ver 'timesfm_2p5' en la lista)


timesfm.__file__: /content/timesfm_repo/src/timesfm/__init__.py
timesfm.__path__: ['/content/timesfm_repo/src/timesfm']
Submódulos disponibles: ['configs', 'torch']


In [ ]:
# Import explícito desde el archivo que encontraste:
from timesfm.timesfm_2p5.timesfm_2p5_torch import TimesFM_2p5_200M_torch

# Usa el mixin de HuggingFace ya implementado (_from_pretrained backend)
model = TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")

# Alternativamente:
# model = TimesFM_2p5_200M_torch()
# model.model.load_checkpoint("/ruta/a/model.safetensors")


config.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/925M [00:00<?, ?B/s]

In [ ]:
import numpy as np
model.compile(
    timesfm.ForecastConfig(
        max_context=1024,
        max_horizon=256,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
    )
)
point_forecast, quantile_forecast = model.forecast(
    horizon=12,
    inputs=[
        np.linspace(0, 1, 100),
        np.sin(np.linspace(0, 20, 67)),
    ],  # Two dummy inputs
)
point_forecast.shape  # (2, 12)
quantile_forecast.shape  # (2, 12, 10): mean, then 10th to 90th quantiles.

(2, 12, 10)